In [391]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 가져오기

In [392]:
old_people_population = pd.read_csv('고령화.txt',sep ='\t')
silver_zone = pd.read_csv('전국노인장애인보호구역표준데이터.csv',encoding = 'cp949')
young_zone = pd.read_csv('전국어린이보호구역표준데이터.csv', encoding = 'cp949')
old_welfare = pd.read_csv('노인여가복지시설.txt',sep ='\t')
old_nursing = pd.read_csv('서울특별시 노인요양시설 정보_20180821 (1).csv', encoding='cp949')
young_nursery = pd.read_csv('어린이보육시설.txt',sep = '\t')

## 노인 인구 데이터 전처리

In [393]:
#데이터 확인
# old_people_population.head(5)
# 데이터 정리 - 전체인구+65세이상인구 칼럼만 사용
old_people_population = old_people_population.drop([0,1,2]).reset_index()
old_people_population = old_people_population[old_people_population['동'] == '소계']
old_people_population = old_people_population[['자치구','전체인구','65세이상 인구']]

In [394]:
#str데이터 int로 바꿔주기
old_people_population['전체인구'] = old_people_population['전체인구'].str.replace(',','')
old_people_population['65세이상 인구'] = old_people_population['65세이상 인구'].str.replace(',','')
old_people_population = old_people_population.astype({'전체인구':int,'65세이상 인구':int})

In [395]:
old_people_population.dtypes

자치구         object
전체인구         int32
65세이상 인구     int32
dtype: object

In [396]:
# 노인인구 비율 칼럼 추가
old_people_population['노인비율'] = old_people_population['65세이상 인구']/old_people_population['전체인구']

### 전체인구 vs 노인인구

In [397]:
# 2019년 전체 인구
total_population_2019 = old_people_population.groupby('자치구').sum()[['전체인구','65세이상 인구']].sum()
total_population_2019

전체인구        10010983
65세이상 인구     1485272
dtype: int64

In [398]:
# 2019년 서울시 노인인구 vs 전체인구
old_people_population = old_people_population.set_index('자치구')
old_people_population

,전체인구,65세이상 인구,노인비율
자치구,,,
종로구,161869,27519,0.170008
중구,136488,23275,0.170528
용산구,245185,38791,0.158211
성동구,308979,43947,0.142233
광진구,366972,47902,0.130533
동대문구,363023,59350,0.163488
중랑구,402024,65164,0.162090
성북구,454744,70850,0.155802
강북구,317695,60577,0.190677


In [399]:
old_people_population.sort_values(by=['노인비율'],axis=0,ascending = False)

,전체인구,65세이상 인구,노인비율
자치구,,,
강북구,317695,60577,0.190677
도봉구,335631,58714,0.174936
중구,136488,23275,0.170528
종로구,161869,27519,0.170008
은평구,484546,80738,0.166626
동대문구,363023,59350,0.163488
중랑구,402024,65164,0.162090
서대문구,323171,52074,0.161135
용산구,245185,38791,0.158211


In [401]:
district_old = old_people_population['65세이상 인구']
district_old.sort_values(ascending=False)

자치구
송파구     87334
강서구     84077
노원구     80923
은평구     80738
관악구     75194
강남구     71025
성북구     70850
구로구     65779
중랑구     65164
강동구     63452
동작구     62226
양천구     61264
강북구     60577
동대문구    59350
도봉구     58714
영등포구    57872
서초구     57136
마포구     52429
서대문구    52074
광진구     47902
성동구     43947
용산구     38791
금천구     37660
종로구     27519
중구      23275
Name: 65세이상 인구, dtype: int32

## 노인보호구역, 어린이구역 데이터 전처리

In [324]:
silver_zone.head()

,장소유형코드,대상시설명,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,제한속도,관리기관명,관리기관전화번호,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
0,1,부암1리 경로당,충청남도,금산군,44710,충청남도 금산군 진산면 뜬바우길 12,NaN,36.146628,127.395492,30,충청남도 금산군,041-750-2713,금산경찰서,N,0.0,NaN,2019-12-03,4550000,충청남도 금산군
1,1,"천내1,3리 경로당",충청남도,금산군,44710,충청남도 금산군 제원면 금강로 334,NaN,36.112153,127.570152,30,충청남도 금산군,041-750-2713,금산경찰서,N,0.0,NaN,2019-12-03,4550000,충청남도 금산군
2,1,동곡1리 경로당,충청남도,금산군,44710,충청남도 금산군 제원면 동곡길 28,NaN,36.136288,127.546822,30,충청남도 금산군,041-750-2713,금산경찰서,N,0.0,NaN,2019-12-03,4550000,충청남도 금산군
3,1,태안 노인복지회관,충청남도,태안군,44825,충청남도 태안군 태안읍 후곡로 79,충청남도 태안군 태안읍 남문리 712-13,36.745991,126.301671,30,태안군,041-670-2719,태안경찰서,N,NaN,3,2020-02-07,4620000,충청남도 태안군
4,2,장애인종합복지관,전라남도,광양시,46230,전라남도 광양시 광양읍 대림오성로 117,NaN,34.965189,127.574359,30,전라남도 광양시,061-797-2297,광양경찰서,N,0.0,7~11,2020-03-23,4840000,전라남도 광양시


In [325]:
# 필요한 데이터만 가져오기 - 서울특별시
silver_zone['시도명'].unique()
silver_zone_seoul = silver_zone[silver_zone['시도명']=='서울특별시']
silver_zone_seoul.head(5)

,장소유형코드,대상시설명,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,제한속도,관리기관명,관리기관전화번호,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
243,1,강북노인종합복지관,서울특별시,강북구,11305,서울 강북구 삼양로92길 40,NaN,37.635732,127.018736,30,서울특별시 강북구,02-901-5927,강북경찰서,N,NaN,8~10,2020-06-25,3080000,서울특별시 강북구
244,1,강북노인회관,서울특별시,강북구,11305,서울 강북구 삼양로 280-1,NaN,37.628507,127.018166,50,서울특별시 강북구,02-901-5927,강북경찰서,N,NaN,20,2020-06-25,3080000,서울특별시 강북구
273,1,방배노인종합복지관,서울특별시,서초구,11650,서울특별시 서초구 방배천로 48,서울특별시 서초구 방배동 455-11,37.480706,126.982777,30,서울시 서초구,02-2155-7185,방배경찰서,Y,1.0,1,2020-07-03,3210000,서울특별시 서초구
274,1,양재노인종합복지관,서울특별시,서초구,11650,서울특별시 서초구 강남대로30길 73-7,서울특별시 서초구 양재동 7-44,37.483020,127.040807,30,서울시 서초구,02-2155-7185,서초경찰서,Y,1.0,3,2020-07-03,3210000,서울특별시 서초구
275,1,중앙노인종합복지관,서울특별시,서초구,11650,서울특별시 서초구 서초대로54길 45,서울특별시 서초구 서초동 1666-17,37.492325,127.016812,30,서울시 서초구,02-2155-7185,서초경찰서,Y,1.0,4,2020-07-03,3210000,서울특별시 서초구


In [326]:
#잘못된 데이터 수정 + 칼럼 이름 수정
silver_zone_seoul['시군구명'].unique()
silver_zone_seoul['시군구명'] = silver_zone_seoul['시군구명'].str.replace('서울특별시 노원구','노원구')
silver_zone_seoul = silver_zone_seoul.rename({'시군구명':'자치구'},axis='columns')

C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [327]:
silver_zone_seoul['자치구'].unique()

array(['강북구', '서초구', '노원구', '강서구', '종로구', '영등포구', '중랑구', '관악구', '성동구',
       '양천구', '강남구', '동작구', '구로구', '송파구', '성북구', '용산구', '동대문구', '강동구',
       '은평구', '광진구'], dtype=object)

In [328]:
young_zone.head(5)

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
0,초등학교,장천초등학교,전라남도 영암군 서호면 서호로 332,전라남도 영암군 서호면 장천리 573,34.756033,126.583025,영암군,영암경찰서,Y,4.0,16,2020-03-30,4940000,전라남도 영암군
1,초등학교,구림초등학교,전라남도 영암군 군서면 왕인로 509,전라남도 영암군 군서면 동구림리 93-1,34.763738,126.632244,영암군,영암경찰서,Y,3.0,12,2020-03-30,4940000,전라남도 영암군
2,초등학교,도포초등학교,전라남도 영암군 도포면 줄다리로 117,전라남도 영암군 도포면 덕화리 100-2,34.836549,126.635336,영암군,영암경찰서,Y,3.0,8,2020-03-30,4940000,전라남도 영암군
3,초등학교,독천초등학교,전라남도 영암군 학산면 영산로 58,전라남도 영암군 학산면 독천리 85,34.728316,126.564459,영암군,영암경찰서,Y,3.0,12,2020-03-30,4940000,전라남도 영암군
4,초등학교,학산초등학교,전라남도 영암군 학산면 금계로 193,전라남도 영암군 학산면 금계리 471-1,34.720560,126.596789,영암군,영암경찰서,Y,3.0,8,2020-03-30,4940000,전라남도 영암군


In [329]:
#서울시 데이터만 가져오기
young_zone_seoul = young_zone.loc[young_zone['소재지도로명주소'].str.contains('서울특별시',na=False)]
young_zone_seoul

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
136,유치원,티움유치원,서울특별시 송파구 송파대로36길 32-6,서울특별시 송파구 송파2동 180-18,37.501103,127.115237,서울특별시 송파구청,송파경찰서,Y,1.0,4~10,2020-02-20,3230000,서울특별시 송파구
137,어린이집,예일어린이집,서울특별시 송파구 백제고분로20길 16,서울특별시 송파구 잠실본동 331,37.503428,127.085721,서울특별시 송파구청,송파경찰서,Y,1.0,4,2020-02-20,3230000,서울특별시 송파구
377,유치원,보고유치원,서울특별시 노원구 석계로13길 23-14,서울특별시 노원구 월계1동 435-5,37.619561,127.061381,노원구청,노원경찰서,Y,1.0,5.5~7.9,2020-07-14,3100000,서울특별시 노원구
378,유치원,영은유치원,서울특별시 노원구 동일로213길 78,서울특별시 노원구 상계6동 740,37.648119,127.057571,노원구청,노원경찰서,Y,1.0,19.6~21,2020-07-14,3100000,서울특별시 노원구
379,유치원,아이디유치원,서울특별시 노원구 동일로221길 32,서울특별시 노원구 상계10동 695,37.659169,127.057173,노원구청,노원경찰서,N,0.0,9.8,2020-07-14,3100000,서울특별시 노원구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17925,초등학교,용동초교,서울특별시 노원구 한글비석로1길 81-11,서울특별시 노원구 중계3동 510-7,37.641085,127.069083,노원구청,노원경찰서,Y,2.0,7~20,2020-07-14,3100000,서울특별시 노원구
17957,초등학교,동일초교,서울특별시 노원구 한글비석로 566,서울특별시 노원구 상계8동 887-7,37.664814,127.054172,노원구청,노원경찰서,Y,1.0,20,2020-07-14,3100000,서울특별시 노원구
17958,초등학교,상수초교,서울특별시 노원구 동일로216길 66,서울특별시 노원구 상계7동 755,37.651107,127.064735,노원구청,노원경찰서,Y,2.0,20,2020-07-14,3100000,서울특별시 노원구
18305,초등학교,상경초교,서울특별시 노원구 수락산로 237,서울특별시 노원구 상계8동 621,37.670553,127.051673,노원구청,노원경찰서,Y,1.0,9.7~22,2020-07-14,3100000,서울특별시 노원구


In [330]:
#자치구 칼럼 생성
young_zone_seoul['자치구'] = young_zone_seoul['소재지도로명주소'].str.split(" ").str[1]


C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [331]:
# 잘못된 데이터 수정
young_zone_seoul['자치구'] = young_zone_seoul['자치구'].str.replace('덕릉로62길','도봉구')

C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [332]:
young_zone_seoul[young_zone_seoul['자치구']=='마포구']

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명,자치구


In [333]:
silver_seoul_dist= silver_zone_seoul.groupby('자치구').size()
young_seoul_dist= young_zone_seoul.groupby('자치구').size()

In [334]:
seoul_zone_dist = pd.concat([silver_seoul_dist,young_seoul_dist], axis=1, keys = ['노인','어린이'])

In [335]:
# NaN갑 처리
seoul_zone_dist = seoul_zone_dist.fillna(0)

In [336]:
# 자치구별 노인보호구역 vs 어린이 보호구역
seoul_zone_dist = seoul_zone_dist.astype({'노인':int,'어린이':int})

In [337]:
#마포구 추가
seoul_zone_dist.loc['마포구'] = [0,0]

### 노인 vs 어린이

In [402]:
seoul_zone_dist

,노인,어린이
강남구,13,101
강동구,8,89
강북구,2,40
강서구,15,76
관악구,8,66
광진구,4,80
구로구,2,66
노원구,6,106
동대문구,6,72
동작구,7,0


## 노인인구비율 vs 노인보호구역

In [339]:
populationVszone = pd.concat([district_population['노인비율'], seoul_zone_dist['노인']],axis=1).fillna(0)

In [340]:
populationVszone = populationVszone.astype({'노인':int})

In [341]:
populationVszone

,노인비율,노인
강남구,0.129087,13
강동구,0.144081,8
강북구,0.190677,2
강서구,0.140533,15
관악구,0.145349,8
광진구,0.130533,4
구로구,0.149712,2
금천구,0.149551,0
노원구,0.150610,6
도봉구,0.174936,0


## 노인요양시설/여가시설 데이터 전처리


In [342]:
# 여가시설
old_welfare.head()

,기간,자치구,시설합계,노인복지관,노인복지관.1,경로당,노인교실
0,기간,자치구,시설합계,시설수,종사자수,경로당,노인교실
1,2019,합계,"3,942",85,"2,104","3,466",391
2,2019,종로구,76,3,107,63,10
3,2019,중구,53,2,62,47,4
4,2019,용산구,111,2,42,88,21


In [343]:
old_welfare = old_welfare.drop([0,1])
old_welfare_seoul = old_welfare[['자치구','시설합계']]

In [344]:
old_welfare_seoul= old_welfare_seoul.set_index('자치구')
old_welfare_seoul = old_welfare_seoul.rename({'시설합계':'복지시설'},axis='columns')
old_welfare_seoul.head()

,복지시설
자치구,
종로구,76
중구,53
용산구,111
성동구,179
광진구,108


In [345]:
old_welfare_seoul.shape

(25, 1)

In [289]:
# 요양시설
old_nursing.head()

,번호,사업장명,소재지전체주소,도로명전체주소,개업일자,영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지우편번호,입소정원,자격소유인원수,총인원수,위치정보(X),위치정보(Y),상세영업상태명
0,1,하늘정원실버케어,서울특별시 서대문구 홍제동 361-57번지,NaN,20091130,운영중,NaN,NaN,NaN,NaN,NaN,NaN,18.0,9.0,13.0,194303.954462,453593.870967,운영
1,2,삼성요양원,서울특별시 관악구 신림동,NaN,20160127,운영중,NaN,NaN,NaN,NaN,NaN,NaN,13.0,4.0,5.0,NaN,NaN,운영
2,3,사랑가득 요양원,서울특별시 금천구 독산동 영재빌딩,NaN,20161101,운영중,NaN,NaN,NaN,NaN,NaN,NaN,43.0,NaN,21.0,NaN,NaN,운영
3,4,상계요양원,서울특별시 노원구 상계동,NaN,20170613,운영중,NaN,NaN,NaN,NaN,NaN,NaN,82.0,NaN,48.0,NaN,NaN,운영
4,5,A+도봉재활요양원,서울특별시 도봉구 방학동,NaN,20150521,운영중,NaN,NaN,NaN,NaN,NaN,NaN,41.0,21.0,3.0,NaN,NaN,운영


In [290]:
# 자치구 칼럼 생성하기
old_nursing['자치구'] = old_nursing['소재지전체주소'].str.split(" ").str[1]

In [293]:
# 데이터 확인
old_nursing['자치구'].unique()

array(['서대문구', '관악구', '금천구', '노원구', '도봉구', '종로구', '중랑구', '성북구', '강동구',
       '강서구', '구로구', '은평구', '양천구', '동작구', '송파구', '강북구', '마포구', '부천시',
       '광진구', '서초구', '용인시', '수원시', '용산구', '파주시', '성동구', '군포시', '영등포구',
       '시흥시', '고양시', '강화군', '천안시', '강남구', '춘천시', '동대문구', '중구'],
      dtype=object)

In [298]:
old_nursing_seoul = old_nursing[old_nursing['자치구'].str.contains('구')]

In [301]:
old_nursing_seoul['자치구'].unique()

array(['서대문구', '관악구', '금천구', '노원구', '도봉구', '종로구', '중랑구', '성북구', '강동구',
       '강서구', '구로구', '은평구', '양천구', '동작구', '송파구', '강북구', '마포구', '광진구',
       '서초구', '용산구', '성동구', '영등포구', '강남구', '동대문구', '중구'], dtype=object)

In [348]:
old_nursing_seoul.groupby('자치구').size().sort_values(ascending = False)

자치구
도봉구     33
금천구     30
은평구     27
강서구     26
양천구     22
광진구     22
성북구     21
서대문구    21
노원구     18
강북구     16
구로구     15
강동구     14
관악구     14
서초구     12
강남구     11
종로구     11
중랑구     10
동대문구    10
송파구     10
성동구      8
동작구      7
중구       6
영등포구     5
용산구      5
마포구      4
dtype: int64

In [346]:
district_population['노인비율'].sort_values(ascending = False)

자치구
강북구     0.190677
도봉구     0.174936
중구      0.170528
종로구     0.170008
은평구     0.166626
동대문구    0.163488
중랑구     0.162090
서대문구    0.161135
용산구     0.158211
성북구     0.155802
동작구     0.152175
노원구     0.150610
구로구     0.149712
금천구     0.149551
관악구     0.145349
영등포구    0.144324
강동구     0.144081
성동구     0.142233
강서구     0.140533
마포구     0.135853
양천구     0.132524
서초구     0.131315
광진구     0.130533
강남구     0.129087
송파구     0.127917
Name: 노인비율, dtype: float64

In [370]:
old_welfare_seoul = old_welfare_seoul.astype({'복지시설':int})

In [371]:
old_welfare_seoul = old_welfare_seoul['복지시설']

In [373]:
old_welfare_seoul.sort_values(ascending = False)

자치구
노원구     264
강서구     251
구로구     219
영등포구    205
양천구     197
강남구     195
송파구     190
성북구     190
은평구     180
성동구     179
마포구     173
동작구     156
서초구     152
동대문구    152
도봉구     149
중랑구     142
강동구     138
관악구     134
서대문구    126
강북구     117
용산구     111
광진구     108
금천구      85
종로구      76
중구       53
Name: 복지시설, dtype: int32

# 어린이 보육시설 -> 시설이 몇개 있는지 X

In [374]:
young_nursery.head()

,기간,자치구,보육시설 정원 현황,보육시설 정원 현황.1,보육시설 정원 현황.2,보육시설 정원 현황.3,보육시설 정원 현황.4,보육시설 정원 현황.5,보육시설 정원 현황.6,보육시설 정원 현황.7,보육시설 현원 현황,보육시설 현원 현황.1,보육시설 현원 현황.2,보육시설 현원 현황.3,보육시설 현원 현황.4,보육시설 현원 현황.5,보육시설 현원 현황.6,보육시설 현원 현황.7
0,기간,자치구,계,국공립,사회복지법인,법인·단체 등,민간,가정,부모협동,직장,계,국공립,사회복지법인,법인·단체 등,민간,가정,부모협동,직장
1,2019,서울시,"254,538","96,595","1,890","5,878","91,892","38,168","1,042","19,073","217,444","85,465","1,580","4,872","77,121","33,223",850,"14,333"
2,2019,종로구,"4,881","2,316",100,149,370,70,-,"1,876","3,581","1,747",46,90,294,51,-,"1,353"
3,2019,중구,"4,459","2,033",-,378,344,205,-,"1,499","3,512","1,692",-,310,271,198,-,"1,041"
4,2019,용산구,"5,221","1,970",89,335,"1,136",904,33,754,"4,610","1,776",77,293,"1,013",813,30,608


In [379]:
young_nursery_seoul = young_nursery.drop([0,1])[['자치구',보육시설정원현황]]

In [382]:
young_nursery_seoul['자치구'].unique().shape

(25,)

In [383]:
young_nursery_seoul.groupby('자치구').size()

자치구
강남구     1
강동구     1
강북구     1
강서구     1
관악구     1
광진구     1
구로구     1
금천구     1
노원구     1
도봉구     1
동대문구    1
동작구     1
마포구     1
서대문구    1
서초구     1
성동구     1
성북구     1
송파구     1
양천구     1
영등포구    1
용산구     1
은평구     1
종로구     1
중구      1
중랑구     1
dtype: int64